<h1 style="background-color:#0071BD;color:white;text-align:center;padding-top:0.8em;padding-bottom: 0.8em">
  LDA on Returns
</h1>

This notebook illustrates the core ideas of Latent Dirichlet Allocation on a very minimal corpus. After you have worked through this notebook, you should have understood:
  * A __corpus__ consists of a list of documents.
  * The __vocabulary__ consists of the union of words that we consider relevant in the documents.
  * Each document is represented by the __word counts__ of the words in the vocabulary.
  * A __topic__ is a probability distribution over the vocabulary.
  * The __topic distribution__ gives us the share that each topic has on a given document.
  * Topic distribution times topics is an approximation of the word counts.
  * Very frequent and extremly seldom words do not contribute to the distinction of topics.
  
<p style="background-color:#66A5D1;padding-top:0.2em;padding-bottom: 0.2em" />

## Preliminaries

In [1]:
import re
import itertools

import numpy as np

from sklearn.preprocessing import normalize
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
# Comments in /this/ notebook are meant as invitations to explore alternatives.
# On the first read, you should just ignore all the comments. On a second read
# you might want to add more sentences to the corpus (see cells below).
# So if this is your first read, you should start ignoring comments now.

# If you enlarge the corpus, you might want to enlarge the width of the notebook
# on the screen, to see the tables without line breaks. The two lines below make
# the cells as wide as possible:

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

### Utility functions for printing
Simple printing functions to present values and matrices somewhat harmoniously. 

In [3]:
def print_cell(template, value, threshold):
    if (threshold is None) or (value > threshold): 
        text = template.format(value)
    else:
        size = len(template.format(0))
        text = size * ' '
    print(text, end='')

def print0d(value):
    print('\n=== {} ==='.format(value))

def print1d(template, vector, threshold = None):
    for value in vector: 
        print_cell(template, value, threshold)
    print()
    
def print2d(template, matrix, threshold = None):
    for vector in matrix: 
        print1d(template, vector, threshold)

## Corpus definition and vocabulary creation

Below you find a few sentences that obviously cover two quite distinct topics. They share a common word that has two different meanings. We consider each sentence to be a separate document. Let's see whether Latend Dirichlet Allocation is able to detect that we are looking at two different topics. Notice that the process is unsupervised, i.e. **we never tell the algorithms for any document (sentence) which topic it covers**. The only hint, we will give the algorithm is that it should look out for exactly 2 topics.

In [4]:
corpus = [
    'Investment in Deutsche Bank yields low return.',
    'My investment may return nothing.', 
    'Federer’s return was good, his volley was not.',
    'Return volley, return volley; tennis is boring.',
    'Return on investment is on a ten year high.',
#    'Tennis is for Federer!',
#    'Deutsche Bank may be an investment bank.'
]

n_topics = 2

expected_topics = [0, 0, 1, 1, 0] # Not for training! Just for checking afterwards.
# expected_topics = [0, 0, 1, 1, 0, 1, 0]

In [5]:
bags = []

for document in corpus:
    
    tokens = re.split('[ .!,;’]', document)
    bag    = [token.lower() for token in tokens if len(token) > 3]
    
# Filtering words by length is just a shortcut to filtering words that do not contribute to a topic.
# We call words that we filter out during preprocessing independently of the motivation "stop words". 
# Some words that hardly contribute to a topic are function words. See https://en.wikipedia.org/wiki/Function_word
# You may instead use the following lines:

#    stop_words = ['', 'in', 'my', 'may', 's', 'was', 'his', 'not', 'is', 'on', 'a', 'ten', 'for', 'be', 'an']
#    bag        = [token.lower() for token in tokens if token.lower() not in stop_words]

# Some words that have different meanings are nevertheless refering to the same dimension of
# reality. You could experiment with encoding this previous knowledge by merging such words:

#    bag        = ['hi/lo' if word in {'high', 'low'} else word for word in bag]
    
    bags.append(bag)

print0d('Words per document')
print2d('{:12s}', bags)


=== Words per document ===
investment  deutsche    bank        yields      return      
investment  return      nothing     
federer     return      good        volley      
return      volley      return      volley      tennis      boring      
return      investment  year        high        


In [6]:
vocabulary = dict.fromkeys([word for bag in bags for word in bag])

# We create the vocabulary from the documents by substracting
# You could try to use a reduced vocabulary and see how LDA performs then:

# vocabulary = dict.fromkeys(['investment', 'return', 'federer', 'volley'])
# for bag in bags: bag = [word for word in bag if word in vocabulary]

words = [word for word in vocabulary.keys()]

print0d('Combined vocabulary of all documents')
print1d('{}  ', words)


=== Combined vocabulary of all documents ===
investment  deutsche  bank  yields  return  nothing  federer  good  volley  tennis  boring  year  high  


In [7]:
for key in vocabulary.keys(): vocabulary[key] = 0
word_counts = np.zeros((len(bags), len(words)), dtype=int)

for d, bag in enumerate(bags):
    for w, word in enumerate(words):
        
        count = bag.count(word)
        
        vocabulary[word] += count
        word_counts[d, w] = count

LINE = (len(vocabulary) * 9 + 5) * '~'
print0d('Word counts in the documents'); print(LINE)
print1d('{:>9}', vocabulary.keys()    ); print(LINE)
print2d('{:9d}', word_counts,        0); print(LINE)
print1d('{:9d}', vocabulary.values()  )


=== Word counts in the documents ===
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
investment deutsche     bank   yields   return  nothing  federer     good   volley   tennis   boring     year     high
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        1        1        1        1        1                                                                        
        1                                   1        1                                                               
                                            1                 1        1        1                                    
                                            2                                   2        1        1                  
        1                                   1                                                              1        1
~~~~~~~

## Latent Dirichlet Allocation

In [8]:
lda = LatentDirichletAllocation(n_components = n_topics, learning_method='batch', max_iter=50, n_jobs = -1)

lda.fit(word_counts)

words_in_topics = normalize(lda.components_, norm='l1')

print0d('Words in the topics'); print(LINE)
print1d('{:>9}',   vocabulary.keys()); print(LINE)
print2d('{:9.1f}', lda.components_  ); print(LINE)
print2d('{:9.1%}', words_in_topics  ); print(LINE)


=== Words in the topics ===
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
investment deutsche     bank   yields   return  nothing  federer     good   volley   tennis   boring     year     high
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      3.5      1.5      1.5      1.5      3.4      1.5      0.5      0.5      0.5      0.5      0.5      1.5      1.5
      0.5      0.5      0.5      0.5      3.6      0.5      1.5      1.5      3.5      1.5      1.5      0.5      0.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    19.0%     8.1%     8.1%     8.1%    18.6%     8.1%     2.7%     2.7%     2.7%     2.7%     2.7%     8.1%     8.1%
     3.1%     3.0%     3.0%     3.0%    21.6%     3.1%     9.0%     9.0%    21.0%     9.0%     9.0%     3.1%     3.1%
~~~~~~~~~~~

In [9]:
topics_in_corpus = lda.transform(word_counts)

print0d('Topics in the documents')
print1d('Topic{:2d}  ', range(n_topics)      )
print2d('{:7.0%}  ',    topics_in_corpus, 0.5)


=== Topics in the documents ===
Topic 0  Topic 1  
    91%           
    85%           
             89%  
             92%  
    89%           


In [10]:
actual_topics = np.argmax(topics_in_corpus, axis = 1)
topics_are_as_expected = any([
    np.array_equal(actual_topics, np.array(permutation)[expected_topics]) 
        for permutation in itertools.permutations(range(n_topics))
])
print('\\(^_^)/ The algorithm found the topic distribution that we expected. Nice!' if topics_are_as_expected 
    else '<(>_<)> NOOO!!! On this run the algorithm did NOT see the same topic distribution in the corpus as humans do.')

\(^_^)/ The algorithm found the topic distribution that we expected. Nice!


In [11]:
length_in_corpus = [len(bag) for bag in bags]
estimated_word_counts = np.diag(length_in_corpus).dot(topics_in_corpus).dot(words_in_topics)

print0d('Actual words in documents compared to estimated words in documents'); print(LINE)
print1d('{:>9}',   words                       ); print(LINE)
print2d('{:9d}',   word_counts,           0    ); print(LINE)
print2d('{:9.1f}', estimated_word_counts, 0.334)


=== Actual words in documents compared to estimated words in documents ===
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
investment deutsche     bank   yields   return  nothing  federer     good   volley   tennis   boring     year     high
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        1        1        1        1        1                                                                        
        1                                   1        1                                                               
                                            1                 1        1        1                                    
                                            2                                   2        1        1                  
        1                                   1                                          

In [12]:
def topic_description(words, probabilities):

    cumulated = 0
    description = ''
    
    for w in np.argsort(probabilities)[::-1]:

        probability = probabilities[w]
        description += words[w]  + ','
        
        if (cumulated < 1/3 <= cumulated + probability) or (cumulated < 4/5 <= cumulated + probability):
            description += '  '
        
        cumulated += probability
    
    return description.rstrip(' ').rstrip(',')

descriptions = [topic_description(words, probabilities) for probabilities in words_in_topics]

print0d('Short descriptions for the topics')
print2d('{}', descriptions)


=== Short descriptions for the topics ===
investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deutsche


In [13]:
for document, probabilities in zip(corpus, topics_in_corpus):

    print('\n"{}"'.format(document))
    
    for probability, description in zip(probabilities, descriptions):
        print('{}  {:3.0%} {:}'.format('X' if probability > 0.5 else '-', probability, description))


"Investment in Deutsche Bank yields low return."
X  91% investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
-   9% return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deutsche

"My investment may return nothing."
X  85% investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
-  15% return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deutsche

"Federer’s return was good, his volley was not."
-  11% investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
X  89% return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deutsche

"Return volley, return volley; tennis is boring."
-   8% investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
X  92% return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deuts

## Lattice of the "Sentence uses word" relation

Have you been suspicious of whether we actually need a probabilistic approach to distinguish these few documents? If yes, you were right. The lattice below illustrates, which document contains which word. A document contains a word if you can reach a word starting from the document by following lines upwards. As you see, we could just ignore "return" as all documents contain this word. The presence of "investment" or "volley" separates the corpus into two. The remaining words are then just specific to each of the documents.

<img src='images/lda-on-returns-word-use-in-5-sentences.PNG' style='width:60%'/>

To distinguish the topics words that appear in (almost) all documents are as uninteresting as words that appear only in (not much more than) one document. For a larger corpus you might thus filter words that are in many documents (we had good results with an upper bound of 50%, but even lower values might work) as well as very few documents (we had reasonable good results with a lower bound of 3 documents given a corpus of about 1000 documents, but even higher values might work).

The lattice was created with "The Concept Explorer" version 1.3, created by Serhiy A. Yevtushenko, available at http://conexp.sourceforge.net/.

### Lattice of the "Sentence uses word" relation, given two more sentences

So, it is time to increase the corpus a bit. Scroll back to the top and include the given two more sentences. The lattice below demonstrates that an analysis based just on set theory becomes harder.

<img src='images/lda-on-returns-word-use-in-7-sentences.PNG' style='width:60%'/>

There is now a sentence that does not include "return" and while "investment" would still be enough to identify the one topic "volley" is not part of all sentences belonging to the other topic ("tennis"). Each of the three sentences of the topic "tennis" contains two of the three words "tennis", "federer", "volley". So, one could say that the probability of a document containing one of these words given the document belongs to the topic "tennis" is 2/3.

The actual probabilistic model behind LDA is slightly different. It assumes that the whole corpus is generated as follows: The topic shares are distributed according to a Dirichlet distribution (https://en.wikipedia.org/wiki/Dirichlet_distribution). Within each topic each word in the vocabulary has a certain probability (https://en.wikipedia.org/wiki/Multinomial_distribution). For each document first the topic shares are randomly picked. Then for each word in the document first a topic is randomly picked according to the topic shares and then the actual word is randomly picked according to the probabilities of the words in the topic. A good overview can be found in: 
Blei, D. M. (2012). Probabilistic Topic Models. Communications of the ACM, 55(4), 77–84 (https://doi.org/10.1145/2133806.2133826).

## Applying the learned topic model to new sentences

We had trained the probabilities of the words within the topics. These can be used to estimate the topic shares within new sentences that were not used for training. Here are a few. Obviously they were created to challenge the model. See how the model performs. You might want to compare the results for a model fitted to the corpus of 5 and for the corpus of 7. The two more sentences seem to make quite a difference. Try for yourself:

In [14]:
more_documents = [
    'I want to return from tennis.',
    'Investment is nothing boring.',
    'Tennis is nothing boring.',
    'After a good return, I return to volley.',
    'This is my year of high investment.',
    'Federer\'s investment is at Deutsche Bank',
    'Playing tennis for hours is quite an investment.',
]

more_bags = [[token.lower() for token in re.split('[ .!,;’]', document) if len(token) > 3] for document in more_documents]
more_word_counts = [[bag.count(word) for word in words] for bag in more_bags]

topics_in_more_documents = lda.transform(more_word_counts)

for document, probabilities in zip(more_documents, topics_in_more_documents):
    print('\n"{}"'.format(document))
    for probability, description in zip(probabilities, descriptions):
        print('{} {:3.0%} {:}'.format('X ' if probability > 0.5 else '- ', probability, description))     

estimated_word_counts = np.diag([len(bag) for bag in more_bags]).dot(topics_in_more_documents).dot(words_in_topics)

print()
print0d('Actual words in documents compared to estimated words in documents'); print(LINE)
print1d('{:>9}',   words                       ); print(LINE)
print2d('{:9d}',   more_word_counts,      0    ); print(LINE)
print2d('{:9.1f}', estimated_word_counts, 0.334)


"I want to return from tennis."
-  20% investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
X  80% return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deutsche

"Investment is nothing boring."
X  66% investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
-  34% return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deutsche

"Tennis is nothing boring."
-  32% investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
X  68% return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deutsche

"After a good return, I return to volley."
-  12% investment,return,  yields,bank,deutsche,high,year,nothing,  good,federer,volley,boring,tennis
X  88% return,volley,  boring,tennis,good,federer,nothing,  investment,high,year,yields,bank,deutsche

"This is my year of high investment."
X  87%

## Where to go from here

Have you played around enough with this notebook already? You should have tried some of the suggested variations. Here are some ideas about where to continue from here. They are in no particular order. Choose depending on your interest.

### Study the maths

With a good background in probability theory you might study the original paper: Blei, D. M., Ng, A. Y., Jordan, M. I. (2003). Latent Dirichlet Allocation. Journal of Machine Learning Research, 3, 993–1022. (https://dl.acm.org/citation.cfm?id=944919.944937)

The following lecture about Pattern Recognition by Prof. Christian Bauckhage might be helpful. While none of the lectures  addresses Latent Dirichlet Allocation directly, they provide important background knowledge.
  * [Pattern Recognition, Lecture 09, Maximum Likelihood Techniques](https://www.researchgate.net/project/lectures-on-pattern-recognition/update/5cd1b0f33843b0b98251ae33)
  * [Pattern Recognition, Lecture 10, Bayesian inference](https://www.researchgate.net/project/lectures-on-pattern-recognition/update/5cdfd7653843b0b982536a37)
  * [Pattern Recognition, Lecture 14, Expectation Maximization](https://www.researchgate.net/project/lectures-on-pattern-recognition/update/5cee3bd63843b0b98254cfd9), Expectation Maximization is the algorithm behind Latent Dirichlet Allocation in the original paper. The EM-Algorithm is covered in this lecture. But! The LDA covered in this lecture is Fishers Latent Discriminant Analysis. 

We have prepared two notebooks illustrating the EM-Algorithm for Gaussian Mixture Models:
  * [Expectation Maximization for Gaussian Mixture Models](https://nbviewer.jupyter.org/github/p3ml/recipes/blob/master/Expectation%20Maximization%20for%20Gaussian%20Mixture%20Models.ipynb)
  * [EM-Algorithm for GMMs - Detailed Visualization and Playground](https://nbviewer.jupyter.org/github/p3ml/recipes/blob/master/EM-Algorithm%20for%20GMMs%20-%20Detailed%20Visualization%20and%20Playground.ipynb) 

### Work on a larger corpus

Why let a machine do something that you can do better? You should have a look at some data where it is helpful that machines are faster than us or that they can process larger amounts of data than a human. We prepared an experiment here:

https://p3ml.github.io/#notebooks-about-latent-dirichlet-allocation

### Work with other implementations

As the original paper about Latent Dirichlet Allocation was already published in 2003, it comes at no surprise that there is already a lot of useful material available. Here are two implementations and an exciting visualization:

  * scikit-learn's LatentDirichletAllocation: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html used above, you may leave the counting to https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html.
  * Radim Řehůřek's gensim: https://radimrehurek.com/gensim/models/ldamodel.html
  * pyLDAvis - Python library for interactive topic model visualization: https://github.com/bmabey/pyLDAvis

A ready made suite to apply LDA on a corpus and to navigate the corpus based on the topic model is available here:

  * InPhO Topic Explorer: https://www.hypershelf.org/
  * See for example the model created for the "Stanford Encyclopedia of Philosophy (Spring 2018)" with 80 topics focusing on the document titled "Turing Machine": https://www.hypershelf.org/sep/80/?doc=turing-machine

### Refactor this notebook and compare the options side by side
If you have some experience in software development, you might not be satisfied with having to change notebook and to re-execute the whole notebook to compare results. You might want to wrap the steps into functions with parameters instead of working on global variables in top level statements. With a bit preparation it should be possible to compare the performance of the model that were fitted to different corpora after different pre-processing side by side.

<table style="width:100%">
  <tr>
      <td colspan="1" style="text-align:left;background-color:#0071BD;color:white">
        <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">
            <img alt="Creative Commons License" style="border-width:0;float:left;padding-right:10pt"
                 src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" />
        </a>
        &copy; D. Speicher<br/>
        Licensed under a 
        <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/" style="color:white">
            CC BY-NC 4.0
        </a>.
      </td>
      <td colspan="2" style="text-align:left;background-color:#66A5D1">
          <b>Acknowledgments:</b>
          This material was prepared within the project
          <a href="http://www.b-it-center.de/b-it-programmes/teaching-material/p3ml/" style="color:black">
              P3ML
          </a> 
          which is funded by the Ministry of Education and Research of Germany (BMBF)
          under grant number 01/S17064. The authors gratefully acknowledge this support.
      </td>
  </tr>
</table>